Task 5

In [2]:
!pip install langgraph.checkpoint.sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 9.1 MB/s eta 0:00:00


In [ ]:
# final_agent_graph_auto_execute_on_resume_with_clean_display.py
import json
import ast
import math
import operator as op
import traceback
import inspect
import time
import random
import re
from typing import Any, TypedDict, List, Dict, Optional
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver

try:
    import openai
    RateLimitExc = getattr(openai.error, "RateLimitError", None)
except Exception:
    openai = None
    RateLimitExc = None

_ALLOWED_OPERATORS = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
    ast.USub: op.neg,
    ast.UAdd: op.pos,
    ast.Mod: op.mod,
    ast.FloorDiv: op.floordiv,
}
_ALLOWED_NAMES = {k: getattr(math, k) for k in dir(math) if not k.startswith("_")}
_ALLOWED_NAMES.update({"pi": math.pi, "e": math.e})

def safe_eval_expr(expr: str):
    def _eval(node):
        if isinstance(node, ast.Constant):
            if isinstance(node.value, (int, float)):
                return node.value
            raise ValueError("Unsupported constant type")
        if node.__class__.__name__ == "Num":  # legacy
            val = getattr(node, "n", None)
            if isinstance(val, (int, float)):
                return val
            raise ValueError("Unsupported legacy numeric literal")
        if isinstance(node, ast.BinOp):
            op_type = type(node.op)
            if op_type not in _ALLOWED_OPERATORS:
                raise ValueError(f"Operator {op_type} not allowed")
            left = _eval(node.left)
            right = _eval(node.right)
            return _ALLOWED_OPERATORS[op_type](left, right)
        if isinstance(node, ast.UnaryOp):
            op_type = type(node.op)
            if op_type not in _ALLOWED_OPERATORS:
                raise ValueError(f"Unary operator {op_type} not allowed")
            return _ALLOWED_OPERATORS[op_type](_eval(node.operand))
        if isinstance(node, ast.Call):
            if not isinstance(node.func, ast.Name):
                raise ValueError("Only simple function calls allowed")
            fname = node.func.id
            if fname not in _ALLOWED_NAMES:
                raise ValueError(f"Function {fname} not allowed")
            f = _ALLOWED_NAMES[fname]
            args = [_eval(a) for a in node.args]
            return f(*args)
        if isinstance(node, ast.Name):
            name = node.id
            if name in _ALLOWED_NAMES:
                return _ALLOWED_NAMES[name]
            raise ValueError(f"Name {name} not allowed")
        raise ValueError(f"Unsupported AST node: {type(node)}")
    parsed = ast.parse(expr, mode="eval")
    return _eval(parsed.body)

@tool(description="Get the current weather for a single location (simulated).")
def get_weather(location: str) -> str:
    weather_data = {
        "San Francisco": "Sunny, 72°F",
        "New York": "Cloudy, 55°F",
        "London": "Rainy, 48°F",
        "Tokyo": "Clear, 65°F",
    }
    return weather_data.get(location, f"Weather data not available for {location}")

@tool(description="Calculator: accepts a JSON string or raw expression and returns a JSON string with the result.")
def calculator(raw: str) -> str:
    try:
        try:
            data = json.loads(raw)
        except Exception:
            data = {"expr": raw}
        if isinstance(data, dict) and "raw" in data and isinstance(data["raw"], str):
            try:
                nested = json.loads(data["raw"])
                data = nested
            except Exception:
                data = {"expr": data["raw"]}
        if not isinstance(data, dict):
            return json.dumps({"error":"unexpected_payload","payload":str(data)})
        if "expr" in data:
            expr = str(data["expr"])
            val = safe_eval_expr(expr)
            return json.dumps({"result": val, "expression": expr})
        op_name = str(data.get("operation","")).lower()
        if op_name in ("add","sum"):
            a = float(data["a"]); b = float(data["b"]); return json.dumps({"result": a + b})
        if op_name in ("sub","subtract","minus"):
            a = float(data["a"]); b = float(data["b"]); return json.dumps({"result": a - b})
        if op_name in ("mul","multiply"):
            a = float(data["a"]); b = float(data["b"]); return json.dumps({"result": a * b})
        if op_name in ("div","divide"):
            a = float(data["a"]); b = float(data["b"])
            if b == 0: return json.dumps({"error":"division_by_zero"})
            return json.dumps({"result": a / b})
        if op_name in ("sin","cos","tan","sqrt","log","ln","exp"):
            x_expr = data.get("x")
            if x_expr is None: return json.dumps({"error":"missing_argument","message": "x required"})
            x_val = safe_eval_expr(str(x_expr))
            if op_name == "sin": return json.dumps({"result": math.sin(x_val)})
            if op_name == "cos": return json.dumps({"result": math.cos(x_val)})
            if op_name == "tan": return json.dumps({"result": math.tan(x_val)})
            if op_name == "sqrt": return json.dumps({"result": math.sqrt(x_val)})
            if op_name in ("log","ln"): return json.dumps({"result": math.log(x_val)})
            if op_name == "exp": return json.dumps({"result": math.exp(x_val)})
        if op_name == "area_circle":
            r = float(data["radius"]); return json.dumps({"result": math.pi * r * r})
        if op_name == "circumference":
            r = float(data["radius"]); return json.dumps({"result": 2 * math.pi * r})
        if op_name == "area_rectangle":
            w = float(data["width"]); h = float(data["height"]); return json.dumps({"result": w * h})
        if op_name == "perimeter_rectangle":
            w = float(data["width"]); h = float(data["height"]); return json.dumps({"result": 2 * (w + h)})
        if op_name == "area_triangle":
            b = float(data["base"]); h = float(data["height"]); return json.dumps({"result": 0.5 * b * h})
        return json.dumps({"error":"unknown_operation","input":data})
    except Exception as e:
        return json.dumps({"error":"execution_error","message": str(e)})

@tool(description="Count occurrences of a letter in given text (case-insensitive).")
def count_letter(text: str, letter: str) -> str:
    if not isinstance(text, str) or not isinstance(letter, str) or len(letter) == 0:
        return "0"
    return str(text.lower().count(letter.lower()))

@tool(description="Return simple text statistics in JSON (length, words_count, vowels_count, unique_words).")
def text_stats(text: str) -> str:
    s = text or ""
    words = [w for w in (s.strip().split()) if w]
    vowels = sum(1 for ch in s.lower() if ch in "aeiou")
    unique = len(set(w.strip(".,!?;:\"'()[]{}").lower() for w in words if w))
    out = {"length": len(s), "words_count": len(words), "vowels_count": vowels, "unique_words": unique}
    return json.dumps(out)

def normalize_args_obj(function_args: Any, tool_name: str = None):
    if isinstance(function_args, str):
        try:
            return json.loads(function_args)
        except Exception:
            return function_args
    elif isinstance(function_args, dict):
        parsed = {}
        for k, v in function_args.items():
            if isinstance(v, str):
                try:
                    parsed[k] = json.loads(v)
                except json.JSONDecodeError:
                    parsed[k] = v
            else:
                parsed[k] = v
        if "text" in parsed and isinstance(parsed["text"], dict) and "letter" in parsed["text"] and "letter" not in parsed:
            inner_args = parsed.pop("text")
            return {**inner_args, **parsed}
        return parsed
    return function_args

def invoke_tool(tool_obj, function_args, tool_name=None):
    args_obj = normalize_args_obj(function_args, tool_name=tool_name)
    errors = []
    def rec(e):
        errors.append(traceback.format_exc())
    try:
        if hasattr(tool_obj, "run"):
            try:
                return tool_obj.run(args_obj)
            except TypeError:
                try:
                    return tool_obj.run(json.dumps(args_obj))
                except Exception as e:
                    rec(e)
            except Exception as e:
                rec(e)
    except Exception as e:
        rec(e)
    try:
        if hasattr(tool_obj, "invoke"):
            try:
                return tool_obj.invoke(args_obj)
            except TypeError:
                try:
                    return tool_obj.invoke(json.dumps(args_obj))
                except Exception as e:
                    rec(e)
            except Exception as e:
                rec(e)
    except Exception as e:
        rec(e)
    try:
        if callable(tool_obj):
            try:
                if isinstance(args_obj, dict):
                    return tool_obj(**args_obj)
                return tool_obj(args_obj)
            except TypeError:
                try:
                    return tool_obj(json.dumps(args_obj))
                except Exception as e:
                    rec(e)
            except Exception as e:
                rec(e)
    except Exception as e:
        rec(e)
    try:
        sig = None
        try:
            sig = inspect.signature(tool_obj)
        except Exception:
            pass
        if sig is not None and isinstance(args_obj, dict):
            params = sig.parameters
            call_kwargs = {k: v for k, v in args_obj.items() if k in params}
            try:
                return tool_obj(**call_kwargs)
            except Exception as e:
                rec(e)
    except Exception as e:
        rec(e)
    return "[invoke_tool_failed] Could not call tool. Attempts:\n" + "\n\n".join(errors or ["no details"])

llm = ChatOpenAI(model="gpt-4o-mini")
TOOLS = [get_weather, calculator, count_letter, text_stats]
tool_map = {}
for t in TOOLS:
    name = getattr(t, "name", None) or getattr(t, "__name__", None) or getattr(t, "__qualname__", None)
    tool_map[name] = t
llm_with_tools = llm.bind_tools(TOOLS)

SYSTEM_PROMPT_TEXT = (
    "You are a helpful assistant that uses tools. If there is a tool available to perform a calculation, use that tool. "
    "Do not attempt to do the calculation on your own. When calling calculator, pass a JSON string (e.g. '{\"expr\":\"2+2\"}')."
)

def safe_invoke_llm(model_callable, message_objs, max_retries: int = 6, base_delay: float = 0.5):
    attempt = 0
    while True:
        try:
            return model_callable(message_objs)
        except Exception as e:
            attempt += 1
            text = str(e)
            is_rate_limit = False
            if RateLimitExc is not None and isinstance(e, RateLimitExc):
                is_rate_limit = True
            if not is_rate_limit:
                if "RateLimit" in text or "rate limit" in text.lower() or "429" in text:
                    is_rate_limit = True
            if not is_rate_limit:
                raise
            if attempt > max_retries:
                raise RuntimeError(f"LLM request failed after {max_retries} retries due to rate limits. Last error: {e}") from e
            retry_after = None
            try:
                if hasattr(e, "response") and getattr(e.response, "headers", None):
                    headers = e.response.headers
                    retry_after = headers.get("retry-after") or headers.get("Retry-After")
                    if retry_after is not None:
                        retry_after = float(retry_after)
            except Exception:
                retry_after = None
            if retry_after is not None:
                sleep_for = max(retry_after, base_delay * (2 ** (attempt - 1)))
            else:
                sleep_for = base_delay * (2 ** (attempt - 1))
            jitter = random.uniform(0.8, 1.2)
            sleep_for = float(sleep_for) * jitter
            print(f"[RATE LIMIT] attempt {attempt}/{max_retries}. sleeping {sleep_for:.2f}s before retrying...")
            time.sleep(sleep_for)


def serialize_message(msg) -> Dict[str, Any]:
    if msg is None: return {}
    if isinstance(msg, dict): return dict(msg)
    if isinstance(msg, HumanMessage): return {"role":"human", "content": getattr(msg,"content","")}
    if isinstance(msg, SystemMessage): return {"role":"system", "content": getattr(msg,"content","")}
    if isinstance(msg, ToolMessage): return {"role":"tool", "content": getattr(msg,"content",""), "tool_call_id": getattr(msg,"tool_call_id", None)}
    content = getattr(msg,"content", None) or getattr(msg,"text", None) or str(msg)
    tool_calls = getattr(msg,"tool_calls", None)
    serialized_tool_calls = None
    if tool_calls:
        calls_list = []
        for tc in tool_calls:
            if isinstance(tc, dict):
                calls_list.append({"name": tc.get("name"), "args": tc.get("args"), "id": tc.get("id")})
            else:
                calls_list.append({"name": getattr(tc,"name",None), "args": getattr(tc,"args",None), "id": getattr(tc,"id",None)})
        serialized_tool_calls = calls_list
    out = {"role":"assistant", "content": content}
    if serialized_tool_calls is not None: out["tool_calls"] = serialized_tool_calls
    return out

def deserialize_messages_for_model(serialized: List[Dict[str, Any]]) -> List[Any]:
    """
    Build model payload.
    - system/human -> direct mapping
    - assistant -> SystemMessage (preserve assistant content)
    - tool -> SystemMessage containing the tool output (so the LLM sees tool results)
    """
    out = []
    for m in serialized:
        role = m.get("role")
        content = m.get("content","")
        if role == "system":
            out.append(SystemMessage(content=content))
        elif role == "human":
            out.append(HumanMessage(content=content))
        elif role == "assistant":
            out.append(SystemMessage(content=content))
        elif role == "tool":
            out.append(SystemMessage(content=f"[tool_result] {content}"))
        else:
            continue
    return out

def clean_assistant_content(s: str) -> str:
    """Convert simple LaTeX fragments to readable plain text for terminal printing.

    - Removes LaTeX inline math delimiters \( \) and $ $
    - Converts \frac{a}{b} -> (a/b)
    - Converts common macros: \pi -> π, \sin -> sin, \cos -> cos, \tan -> tan
    - Removes stray backslashes otherwise.
    """
    if not s:
        return s

    out = s

    out = re.sub(r'\\\(|\\\)|\\\\\(|\\\\\)', '', out)
    out = re.sub(r'\$(.*?)\$', r'\1', out)
    def _frac_to_div(m):
        a = m.group(1)
        b = m.group(2)
        return f"({a}/{b})"
    out = re.sub(r'\\frac\{([^{}]+)\}\{([^{}]+)\}', _frac_to_div, out)
    macros = {
        r'\\pi': 'π',
        r'\\sin': 'sin',
        r'\\cos': 'cos',
        r'\\tan': 'tan',
        r'\\sqrt': 'sqrt',
        r'\\left': '',
        r'\\right': '',
        r'\\,': ' ',
        r'\\;': ' ',
    }
    for pattern, repl in macros.items():
        out = re.sub(pattern, repl, out)

    out = out.replace('\\', '')

    out = re.sub(r'\s+', ' ', out).strip()
    out = re.sub(r'\(\s+', '(', out)
    out = re.sub(r'\s+\)', ')', out)

    return out

class AgentState(TypedDict, total=False):
    user_input: str
    should_exit: bool
    print_trace: bool
    chat_history: List[Dict[str, Any]]        
    llm_response: Optional[Dict[str, Any]]    

def create_graph():
    def get_user_input(state: AgentState) -> dict:
        print("\n" + "="*50)
        print("Enter your text (or 'quit' to exit):")
        print("="*50)
        print("\n> ", end="")
        user_input = input()
        if user_input.lower() in ['quit','exit','q']:
            print("Goodbye!"); return {"user_input": user_input, "should_exit": True}
        if user_input.lower() == "verbose":
            print("[TRACE]: verbose"); return {"user_input": user_input, "should_exit": False, "print_trace": True}
        if user_input.lower() == "quiet":
            return {"user_input": user_input, "should_exit": False, "print_trace": False}
        chat_history = state.get("chat_history")
        if not chat_history:
            chat_history = [{"role":"system","content": SYSTEM_PROMPT_TEXT}]
        chat_history = list(chat_history) + [{"role":"human","content": user_input}]
        return {"user_input": user_input, "should_exit": False, "chat_history": chat_history}

    def call_llm(state: AgentState):
        serialized_history = state.get("chat_history", [{"role":"system","content":SYSTEM_PROMPT_TEXT}])

        # Truncate history to prevent prompt/token explosion
        KEEP_LAST = 18
        if isinstance(serialized_history, list) and len(serialized_history) > KEEP_LAST + 1:
            head = [m for m in serialized_history[:1] if m.get("role") == "system"]
            tail = serialized_history[-KEEP_LAST:]
            truncated_history = head + tail
        else:
            truncated_history = serialized_history

        message_objs = deserialize_messages_for_model(truncated_history)
        if not message_objs or not isinstance(message_objs[-1], HumanMessage):
            message_objs.append(HumanMessage(content=state.get("user_input","")))

        # Use safe_invoke_llm to handle rate limits
        response = safe_invoke_llm(llm_with_tools.invoke, message_objs)
        response_serialized = serialize_message(response)
        # Persist full (untruncated) history + new assistant message to keep canonical history
        new_chat_history = list(serialized_history) + [response_serialized]
        return {"llm_response": response_serialized, "chat_history": new_chat_history}

    def route_user_input(state: AgentState):
        if state.get("should_exit", False):
            if state.get("print_trace", False): print("[TRACE] Routing: exiting")
            return END
        user_input = state.get("user_input","")
        if user_input == "" or user_input.lower() in ["verbose","quiet"]:
            return "get_user_input"
        if state.get("print_trace", False): print("[TRACE] Routing: calling LLM...")
        return "call_llm"

    def route_after_llm(state: AgentState):
        if state.get("should_exit", False):
            if state.get("print_trace", False): print("[TRACE] Routing: exiting")
            return END
        last_message = state.get("llm_response", {})
        if last_message and isinstance(last_message, dict) and last_message.get("tool_calls"):
            return "call_tools"
        return "print_response"

    def call_tools(state: AgentState):
        last_message = state.get("llm_response", {}) or {}
        tool_calls = last_message.get("tool_calls", []) if isinstance(last_message, dict) else []
        serialized_history = state.get("chat_history", [])
        tool_results_msgs = []
        for tc in tool_calls:
            fname = tc.get("name"); fargs = tc.get("args"); fid = tc.get("id")
            tool_obj = tool_map.get(fname)
            if tool_obj is None:
                result = f"Error: Unknown function {fname}"
            else:
                result = invoke_tool(tool_obj, fargs, tool_name=fname)
            tm = {"role":"tool", "content": str(result), "tool_call_id": fid}
            tool_results_msgs.append(tm)
            if state.get("print_trace", False):
                print(f"[TRACE] tool {fname} args {fargs} -> {result}")
        new_chat_history = list(serialized_history) + tool_results_msgs
        return {"chat_history": new_chat_history, "llm_response": None}

    def print_response(state: AgentState) -> dict:
        chat_history = state.get("chat_history", [])
        last_assistant = None
        for m in reversed(chat_history):
            if m.get("role") == "assistant":
                last_assistant = m; break
        if not last_assistant:
            last_assistant = state.get("llm_response")
        if last_assistant:
            content = last_assistant.get("content", "")
            pretty = clean_assistant_content(content)
            print("Assistant:", pretty)
        else:
            print("Assistant: (no content)")
        return {}

    graph_builder = StateGraph(AgentState)
    graph_builder.add_node("get_user_input", get_user_input)
    graph_builder.add_node("call_llm", call_llm)
    graph_builder.add_node("call_tools", call_tools)
    graph_builder.add_node("print_response", print_response)
    graph_builder.add_edge(START, "get_user_input")
    graph_builder.add_edge("print_response", "get_user_input")
    graph_builder.add_edge("call_tools", "call_llm")
    graph_builder.add_conditional_edges("call_llm", route_after_llm, {"call_tools":"call_tools", "print_response":"print_response"})
    graph_builder.add_conditional_edges("get_user_input", route_user_input, {"get_user_input":"get_user_input","call_llm":"call_llm", END: END})
    return graph_builder

def find_last_assistant_with_tool_calls(chat: List[Dict[str,Any]]):
    for i in range(len(chat)-1, -1, -1):
        m = chat[i]
        if m.get("role") == "assistant" and m.get("tool_calls"):
            return i, m
    return None, None

def save_graph_image(graph, filename="lg_graph.png"):
    try:
        png_data = graph.get_graph(xray=True).draw_mermaid_png()
        with open(filename, "wb") as f:
            f.write(png_data)
        print(f"Graph image saved to {filename}")
    except Exception as e:
        print(f"Could not save graph image: {e}")
        print("Mermaid diagram (copy to renderer):\ngraph TD\n    START --> get_user_input\n    get_user_input --> call_llm\n    call_llm -->|tool_calls| call_tools\n    call_llm -->|no tools| print_response\n    call_tools --> call_llm\n    print_response --> get_user_input\n")

def main():
    print("="*50)
    print("Starting conversation (LangGraph persistent + auto-execute pending tools on resume)...")
    print("="*50,"\n")
    graph_builder = create_graph()
    print("Graph created successfully!\n")
    with SqliteSaver.from_conn_string("checkpoints.db") as checkpointer:
        graph = graph_builder.compile(checkpointer=checkpointer)
        thread_id = "workflow_1"
        config = {"configurable": {"thread_id": thread_id}}
        current_state = graph.get_state(config)
        print("Saving graph visualization...")
        save_graph_image(graph)
        initial_state: AgentState = {
            "user_input":"", "should_exit": False, "print_trace": False,
            "chat_history": [{"role":"system","content": SYSTEM_PROMPT_TEXT}],
            "llm_response": None
        }

        if current_state.next:
            next_raw = current_state.next
            try:
                next_node = next_raw[0] if isinstance(next_raw, (list,tuple)) else next_raw
            except Exception:
                next_node = str(next_raw)
            risky_nodes = ("call_llm","call_tools")
            if next_node in risky_nodes:
                persisted_values = current_state.values or {}
                preserved_chat = list(persisted_values.get("chat_history", initial_state.get("chat_history")) or [])
                idx, assistant_msg = find_last_assistant_with_tool_calls(preserved_chat)
                executed_any = False
                if idx is not None and assistant_msg:
                    pending_tool_calls = assistant_msg.get("tool_calls", [])
                    pending_ids = [tc.get("id") for tc in pending_tool_calls if tc.get("id") is not None]
                    already_present_ids = set()
                    for m in preserved_chat[idx+1:]:
                        if m.get("role") == "tool" and m.get("tool_call_id") in pending_ids:
                            already_present_ids.add(m.get("tool_call_id"))
                    to_execute = [tc for tc in pending_tool_calls if tc.get("id") not in already_present_ids]
                    if to_execute:
                        print(f"⟳ Executing {len(to_execute)} pending tool call(s).")
                        tool_results = []
                        for tc in to_execute:
                            fname = tc.get("name"); fargs = tc.get("args"); fid = tc.get("id")
                            tool_obj = tool_map.get(fname)
                            if tool_obj is None:
                                result = f"Error: Unknown function {fname}"
                            else:
                                result = invoke_tool(tool_obj, fargs, tool_name=fname)
                            print(f"   -> tool {fname} returned: {result}")
                            tool_results.append({"role":"tool","content": str(result), "tool_call_id": fid})
                        new_chat = preserved_chat[:idx+1] + tool_results + preserved_chat[idx+1:]
                        preserved_chat = new_chat
                        executed_any = True
                    else:
                        print("   No missing tool results to execute (they are already present).")
                else:
                    print("   No assistant message with pending tool_calls was found in history.")
                    i = len(preserved_chat)-1
                    removed = []
                    while i >=0 and preserved_chat[i].get("role") == "tool":
                        removed.insert(0, preserved_chat.pop(i))
                        i -= 1
                    if removed:
                        print("   Removed trailing orphan 'tool' messages during fallback repair:")
                        for m in removed: print("    -", m.get("role"), ":", m.get("content"))

                N = 10
                print("\n--- History after auto-execute/repair (most recent last; up to last {} turns) ---".format(N))
                for m in (preserved_chat or [])[-N:]:
                    print(f"{m.get('role')}: {m.get('content')}")
                print("--- end history ---\n")

                safe_state = {
                    "user_input": "",
                    "should_exit": False,
                    "print_trace": persisted_values.get("print_trace", False),
                    "chat_history": preserved_chat,
                    "llm_response": None
                }
                print("Resuming at get_user_input with repaired/executed history.")
                graph.invoke(safe_state, config=config)
            else:
                print("Safe to resume where we left off.")
                graph.invoke(None, config=config)
        else:
            print("Starting new workflow...")
            graph.invoke(initial_state, config=config)

if __name__ == "__main__":
    main()

<>:373: SyntaxWarning: invalid escape sequence '\('
<>:373: SyntaxWarning: invalid escape sequence '\('
/tmp/ipython-input-4057578653.py:373: SyntaxWarning: invalid escape sequence '\('
  - Removes LaTeX inline math delimiters \( \) and $ $


Starting conversation (LangGraph persistent + auto-execute pending tools on resume)...

Graph created successfully!

Saving graph visualization...
Graph image saved to lg_graph.png
Starting new workflow...

Enter your text (or 'quit' to exit):

> What is (2*3)+ 5?
Assistant: The result of (2*3) + 5 is 11.

Enter your text (or 'quit' to exit):

> What is sin(pi/2)?
Assistant: The result of sin(pi/2) is 1.0.

Enter your text (or 'quit' to exit):

> quit
Goodbye!
